In [1642]:
import pyodbc
import pandas as pd

symbol = 'intc'
conn = pyodbc.connect(
    "DRIVER={SQL Server};SERVER=lenovo-desktop;DATABASE=Qihuo;UID=samtsql;PWD=F(W}q:TsyK,7^+>`P28e79s#Uc5n")
query = f"""select m1.date as 'Date', m1.[Close] as 'Market', m2.[close] as 'Stock'  from MinuteQuote m1
inner join MinuteQuote m2 on m1.Date = m2.date
where m1.Date > '2002-04-08 01:20:00' and m1.Contract = 'ym' and m2.Contract = '{symbol}'
order by m1.date"""

df = pd.read_sql(query, conn)
df.set_index('Date', inplace=True)


c:\Users\SHSIEH\Documents\ProjectsFuture\ML\project-walkthroughs\stock\stock_env\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [1643]:
df["StockReturn"] = df.rolling(2).apply(lambda x: x.iloc[1] / x.iloc[0] - 1)["Stock"]
df["MarketReturn"] = df.rolling(2).apply(lambda x: x.iloc[1] / x.iloc[0] - 1)["Market"]
df["OutPerform"] = df["StockReturn"] - df["MarketReturn"]
df["Target"] = (df.apply(lambda x: x > 0)["OutPerform"]).astype(int)

# df.plot.line(y="OutPerform", use_index=True)

In [1644]:
predictors = []
for i in range(12):
    df[f'OutPerform{pow(2,i)*5}'] = df['OutPerform'].rolling(pow(2,i)).sum()
    predictors.append(f'OutPerform{pow(2,i)*5}')
    df[f'StockReturn{pow(2,i)*5}'] = df['StockReturn'].rolling(pow(2,i)).sum()
    predictors.append(f'StockReturn{pow(2,i)*5}')


In [1645]:
prev = df.copy()
prev = prev.shift(1)
data = df[["OutPerform","Target"]]


In [1646]:
data = data.join(prev[predictors])
data = data.copy().dropna()


In [1647]:
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.metrics import precision_score

model = RandomForestClassifier(n_estimators=100, min_samples_split=1200, random_state=1)


In [1648]:
def predict(train, test, predictors, model):
    model.fit(train[predictors], train["Target"])
    preds = model.predict(test[predictors])
    preds = pd.Series(preds, index=test.index, name="Predictions")
    combined = pd.concat([test["Target"], preds], axis=1)
    return combined

In [1649]:
def backtest(data, model, predictors, start=50000, step=5000):
    all_predictions = []
    loop = 0
    for i in range(start, data.shape[0], step):
        train = data.iloc[loop * step:i].copy()
        test = data.iloc[i:(i+step)].copy()
        predictions = predict(train, test, predictors, model)
        all_predictions.append(predictions)
        loop = 0

    return pd.concat(all_predictions)


In [1651]:
data.shape[0]

396531

In [1652]:
predictions = backtest(data, model, predictors)

KeyboardInterrupt: 

In [ ]:
print(predictions)

                     Target  Predictions
Date                                    
2004-11-23 08:15:00       0            0
2004-11-23 08:20:00       1            1
2004-11-23 08:25:00       0            0
2004-11-23 08:30:00       0            0
2004-11-23 08:35:00       1            1
...                     ...          ...
2022-05-25 12:30:00       0            1
2022-05-25 12:35:00       0            1
2022-05-25 12:40:00       0            1
2022-05-25 12:45:00       1            0
2022-05-25 12:50:00       0            1

[346446 rows x 2 columns]


In [ ]:
predictions["Predictions"].value_counts()

0    209642
1    136804
Name: Predictions, dtype: int64

In [ ]:
precision_score(predictions["Target"], predictions["Predictions"])

0.5247580480102921

In [ ]:
import pickle
pickle.dump(model, open(f"{symbol}", 'wb'))

In [ ]:
predictions.to_csv(f"{symbol}.csv")

In [ ]:
data

,OutPerform,Target,OutPerform5,StockReturn5,OutPerform10,StockReturn10,OutPerform20,StockReturn20,OutPerform40,StockReturn40,...,OutPerform640,StockReturn640,OutPerform1280,StockReturn1280,OutPerform2560,StockReturn2560,OutPerform5120,StockReturn5120,OutPerform10240,StockReturn10240
Date,,,,,,,,,,,,,,,,,,,,,
2002-05-14 06:30:00,0.013686,1,0.000234,0.001809,0.003768,0.006658,0.002268,0.004238,-0.001814,0.000027,...,0.013131,0.025772,-0.014566,-0.011543,0.031370,0.053529,-0.020951,-0.003713,-0.019852,-0.018991
2002-05-14 06:35:00,0.002994,1,0.013686,0.031908,0.013920,0.033718,0.016427,0.036752,0.013285,0.033741,...,0.024965,0.055829,-0.000622,0.019177,0.044423,0.084805,-0.009929,0.028196,-0.010246,0.006481
2002-05-14 06:40:00,-0.000776,0,0.002994,0.004667,0.016680,0.036576,0.020447,0.043234,0.016332,0.037806,...,0.027039,0.058648,0.002241,0.023845,0.050021,0.090738,-0.010871,0.032267,-0.005626,0.011730
2002-05-14 06:45:00,-0.002264,0,-0.000776,-0.001161,0.002218,0.003506,0.016138,0.037224,0.014876,0.035439,...,0.031003,0.063021,-0.005668,0.013188,0.046775,0.088309,-0.013703,0.029318,-0.007375,0.009987
2002-05-14 06:50:00,-0.000865,0,-0.002264,-0.002907,-0.003040,-0.004068,0.013640,0.032508,0.015908,0.036746,...,0.029004,0.060114,-0.008718,0.010281,0.043685,0.083505,-0.022837,0.019272,-0.009165,0.005336
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05-25 12:30:00,-0.000777,0,-0.000359,-0.000607,-0.000583,-0.000645,-0.002735,-0.003636,-0.003832,0.001040,...,0.007274,0.031973,0.010066,0.055743,-0.000167,-0.006111,-0.023095,-0.032839,0.008175,-0.065793
2022-05-25 12:35:00,-0.000211,0,-0.000777,-0.001026,-0.001135,-0.001633,-0.002925,-0.004510,-0.005960,-0.003712,...,0.006150,0.029771,0.008088,0.052671,-0.000308,-0.007551,-0.023670,-0.032986,0.006730,-0.067947
2022-05-25 12:40:00,-0.000034,0,-0.000211,-0.001331,-0.000988,-0.002356,-0.001571,-0.003002,-0.005161,-0.004096,...,0.004858,0.026913,0.007631,0.050379,0.000378,-0.007676,-0.024053,-0.037321,0.007253,-0.068257


In [ ]:
tempData = data[['OutPerform']].copy()
# tempPredictions = predictions.copy()


In [ ]:
result1 = pd.merge(tempData, predictions, left_index=True, right_index=True)

In [ ]:
from datetime import datetime
result1['year'] = result1.index.strftime('%Y')

In [ ]:
result1.loc[result1['Predictions'] == 1]

,OutPerform,Target_x,Target_y,Predictions,year
Date,,,,,
2004-11-23 08:20:00,0.000615,1,1,1,2004
2004-11-23 08:35:00,0.000286,1,1,1,2004
2004-11-23 08:55:00,0.000450,1,1,1,2004
2004-11-23 09:10:00,-0.000204,0,0,1,2004
2004-11-23 09:20:00,0.000532,1,1,1,2004
...,...,...,...,...,...
2022-05-25 12:25:00,-0.000359,0,0,1,2022
2022-05-25 12:30:00,-0.000777,0,0,1,2022
2022-05-25 12:35:00,-0.000211,0,0,1,2022


In [ ]:
resultsum = result1.groupby('year').sum()
resultmean = result1.groupby('year').mean()
print(resultsum)


      OutPerform  Target_x  Target_y  Predictions
year                                             
2004   -0.032392       984       984          708
2005    0.018705      9598      9598         6985
2006    0.001758      9576      9576         7240
2007    0.169737      9621      9621         7903
2008   -0.036380      9819      9819         8705
2009    0.180802      9720      9720         8108
2010   -0.236903      9719      9719         7951
2011   -0.138931      9668      9668         8031
2012   -0.048931      9661      9661         7642
2013    0.099515      9813      9813         7368
2014    0.158088      9915      9915         7485
2015    0.235904      9930      9930         7896
2016    0.004051      9831      9831         7563
2017    0.109498      9916      9916         7151
2018    0.277940      9923      9923         8163
2019    0.254602      9944      9944         7833
2020    0.294918     10058     10058         8335
2021    0.249661     10035     10035         8188


In [ ]:
print(resultmean)

        OutPerform  Target_x  Target_y  Predictions
year                                               
2004 -1.560300e-05  0.473988  0.473988     0.341040
2005  9.427798e-07  0.483770  0.483770     0.352067
2006  8.930814e-08  0.486486  0.486486     0.367811
2007  8.675567e-06  0.491745  0.491745     0.403936
2008 -1.831200e-06  0.494237  0.494237     0.438164
2009  9.115315e-06  0.490043  0.490043     0.408772
2010 -1.192147e-05  0.489080  0.489080     0.400111
2011 -6.991292e-06  0.486514  0.486514     0.404136
2012 -2.491152e-06  0.491854  0.491854     0.389064
2013  5.035910e-06  0.496584  0.496584     0.372856
2014  7.984253e-06  0.500758  0.500758     0.378030
2015  1.189271e-05  0.500605  0.500605     0.398064
2016  2.038341e-07  0.494716  0.494716     0.380586
2017  5.542219e-06  0.501898  0.501898     0.361948
2018  1.409359e-05  0.503169  0.503169     0.413924
2019  1.285869e-05  0.502222  0.502222     0.395606
2020  1.481480e-05  0.505249  0.505249     0.418697
2021  1.2563